# Reference Guide to Firefly Python Client API
This notebook aims to give a high-level overview of the Firefly Python client API and important methods to get you started. For **full explanation** of each method and its parameter, check [FireflyClient API reference](https://caltech-ipac.github.io/firefly_client/api/firefly_client.FireflyClient.html).


In [40]:
from firefly_client import FireflyClient

# Following imports are just for example data fetching and processing
from astropy.utils.data import download_file
from astropy.io import fits
import numpy as np
import io

## Initialize Firefly Client
Firefly is a web application (with a URL) that can be controlled from Python with a `FireflyClient` object. 

To indicate which Firefly server to talk to from our Python client, we set the environment variable `FIREFLY_URL`. In this notebook, we use a public Firefly server: the IRSA Viewer (https://irsa.ipac.caltech.edu/irsaviewer). However, you can also run a local Firefly server via a [Firefly Docker image](https://hub.docker.com/r/ipac/firefly) and access it at `http://localhost:8080/firefly`.

There are two ways to initialize `FireflyClient` from Python, depending on whether you're running the notebook in JupyterLab or not:

### From Jupyter Lab
Assuming you have `jupyter-firefly-extensions` set up in your environment as explained [here](https://github.com/Caltech-IPAC/jupyter_firefly_extensions/blob/master/README.md), you can use `make_lab_client()` in JupyterLab, which will open the Firefly viewer in a new tab within the Lab.

In [ ]:
# fc = FireflyClient.make_lab_client()

### From Jupyter Notebook (or even a Python shell)
You can use `make_client()` in a Jupyter Notebook (or even a Python shell), which will open the Firefly viewer in a new web browser tab. `make_client()` also allows you to pass the URL directly (other than through environment variable) as the `url` parameter.

In [3]:
# fc = FireflyClient.make_client() # URL is taken from FIREFLY_URL env variable

# URL can be defined explicitly as a parameter
fc = FireflyClient.make_client(url="https://irsa.ipac.caltech.edu/irsaviewer")

## [Optional] Reinitialize Firefly Client
To clean the state of Firefly server (aka remove all displayed components and start fresh), you can reinitialise it. This is specifically helpful if a Python connection with server is already open and re-running `make_lab_client` (or `make_client`) leads to stale state.

In [4]:
fc.reinit_viewer()

{'success': True}

## Show FITS Image in Firefly

Use the `show_fits_image` method of the FireflyClient object to display a FITS image. It can take a local file path, URL, or a file-like object as input via the `file_input` parameter.

### From a URL
You can specify url of any FITS image. Here we use cutout of a WISE all-sky image:

In [6]:
cutout_image_url = 'https://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p1bm_frm/6a/02206a/149/02206a149-w1-int-1b.fits?center=70,20&size=400pix'

In [7]:
cutout_image_plot_id = 'wise-cutout'
fc.show_fits_image(file_input=cutout_image_url,
                   plot_id=cutout_image_plot_id,
                   title='WISE Cutout')

{'success': True}

### From a local file
You can specify path of any local FITS file. Here we download the full WISE all-sky image and use its file path:

In [8]:
full_image_url = 'http://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p1bm_frm/6a/02206a/149/02206a149-w1-int-1b.fits'
full_image_fpath = download_file(full_image_url, cache=True, timeout=120)
full_image_fpath

'/Users/jsinghal/.astropy/cache/download/url/8d0c853f180330d78cea76026057a827/contents'

In [9]:
full_image_plot_id = 'wise-fullimage'
fc.show_fits_image(file_input=full_image_fpath,
                   plot_id=full_image_plot_id, # using a different plot id to not overwrite previous plot
                   title='WISE Full Image')

{'success': True}

### From an in-memory file-like object
You can also use a file-like object (e.g. `BytesIO` stream) instead of a local file path or URL. This is useful when you are working with a FITS file in memory and don't want to write it to disk. 


In [41]:
processed_fits = io.BytesIO()

with fits.open(cutout_image_url) as hdul:
    # Do some processing on FITS image data
    img_data = hdul[0].data
    img_data[:10, :] = img_data[-10:, :] = img_data[:, :10] = img_data[:, -10:] = np.nanmax(img_data)
    img_data = np.sqrt(img_data)

    new_hdu = fits.PrimaryHDU(data=img_data, header=hdul[0].header)
    new_hdu.writeto(processed_fits, overwrite=True)
    processed_fits.seek(0) # to bring reading pointer to the beginning of file

/var/folders/v9/2dzh8dk57glf4lxq5vv50sdm0005p2/T/ipykernel_23034/1845551643.py:7: RuntimeWarning: invalid value encountered in sqrt
  img_data = np.sqrt(img_data)


In [42]:
fc.show_fits_image(file_input=processed_fits,
                   plot_id='wise-processed',
                   title='Processed WISE Cutout')

{'success': True}

## Modify the displayed image(s)
### Panning to a coordinate
To pan the full image to center on a cutout's center 70 +20 EQ-J2000:

In [43]:
fc.set_pan(plot_id=full_image_plot_id, # note we use the plot id we defined above
           x=70, y=20, coord='j2000')

{'success': True}

### Zooming
To zoom into the full image by a factor of 2:

In [44]:
fc.set_zoom(plot_id=full_image_plot_id, factor=2)

{'success': True}

### Aligning multiple images
To align both full image and cutout image by WCS and to lock the WCS matching:

In [45]:
fc.align_images(lock_match=True)

{'success': True}

### Changing the color stretch
Set the stretch for the full image based on IRAF zscale interval with a linear algorithm:

In [46]:
fc.set_stretch(plot_id=full_image_plot_id, stype='zscale', algorithm='linear')

{'success': True,
 'rv_string': '91,1.000000,91,1.000000,NaN,2.000000,44,25,600,120,0,NaN,1.000000'}

## Show Table/Catalogs in Firefly

Use the `show_table` method of the FireflyClient object to display a table/catalog. Similar to `show_fits_image`, it can take a local file path, URL, or a file-like object as input via the same `file_input` parameter.

Here we use a 2MASS catalog (using IRSA VO TAP):

In [47]:
table_url = "http://irsa.ipac.caltech.edu/TAP/sync?FORMAT=IPAC_TABLE&QUERY=SELECT+*+FROM+fp_psc+WHERE+CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',70.0,20.0,0.1))=1"

In [48]:
table_id = '2mass-catalog'
fc.show_table(file_input=table_url, # can also be a local file path or file-like object
              tbl_id=table_id,
              title='2MASS Catalog')

{'success': True}

Note: Displaying a catalog table also caused Firefly to overlay the image with markers by default. The markers can be visible on both "wise-cutout" and "wise-fullimage" since they would include the same field of view. Also we see the active chart and coverage tab as bonus!

## Modify the displayed table


### Sorting by a column
You can sort the table by a column in ascending (ASC) or descending (DESC) order:

In [49]:
fc.sort_table_column(tbl_id=table_id,
                     column_name='j_m', sort_direction='ASC')

{'success': True}

### Filtering the rows
Filters can be specified as an SQL WHERE clause-like string with column names quoted:

In [50]:
fc.apply_table_filters(tbl_id=table_id, # note: we use the table id we defined above
                       filters='"j_m">15 and "j_m"<16 and "j_cmsig"<0.06')

{'success': True}

To remove the filters:

In [51]:
fc.apply_table_filters(tbl_id=table_id, filters='')

{'success': True}

## Plot Chart from the displayed table
We can use the dispalyed table data to plot columns on X and Y axis:

In [52]:
fc.show_xyplot(tbl_id=table_id, xCol='j_m', yCol='h_m-k_m')

{'success': True}

Alternatively, more generic method show_chart() can be used to create the same plot which accepts a lot of configuration parameters as [Plotly data structure](https://plotly.com/javascript/reference/index/):

In [53]:
trace0 = {'tbl_id': table_id, 'x': "tables::j_m", 'y': "tables::h_m-k_m", 
          'type' : 'scatter', 'mode': 'markers',
          'marker': dict(size=4, color='red', opacity=0.6)}
layout = {'title': '2MASS color-mag',
           'xaxis': dict(title='J'), 'yaxis': dict(title='H - K')}

status = fc.show_chart(data=[trace0], layout=layout)

## Plot Region overlays on the displayed image

[ds9-style regions](https://ds9.si.edu/doc/ref/region.html) can be overlaid on the image displayed by Firefly:

In [57]:
sources_box = "icrs;box 70d 20d 12' 12' 0d"
image_regions_id = 'wise-regions'
fc.overlay_region_layer(region_data=sources_box,
                        title='WISE all-sky Annotations', 
                        region_layer_id=image_regions_id)

{'success': True}

Add region data to an existing region layer:

In [55]:
ra, dec = 69.992614, 20.088488
point_source = f'icrs;point {ra}d {dec}d # point=cross 25 text={{Source A}}'
fc.add_region_data(region_data=point_source,
                   region_layer_id=image_regions_id) # note: we use same region_layer_id

{'success': True}

To remove region layer:

In [59]:
fc.delete_region_layer(image_regions_id)

{'success': True}

## Show HiPS image

In [60]:
hips_url = 'http://alasky.u-strasbg.fr/DSS/DSSColor'
hips_plot_id = 'hips-img-id'
target = '70;20;EQ_J2000'

fc.show_hips(plot_id=hips_plot_id, 
             hips_root_url=hips_url,
             Title='HiPS-DSS', 
             WorldPt=target)

{'success': True}

## Show 3-color image

In [61]:
# [W4, W2, W1] from WISE all-sky as [R, G, B] 
rgb_urls = ['https://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p3am_cdd/07/0704/0704p196_ab41/0704p196_ab41-w4-int-3.fits',
            'https://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p3am_cdd/07/0704/0704p196_ab41/0704p196_ab41-w2-int-3.fits',
            'https://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p3am_cdd/07/0704/0704p196_ab41/0704p196_ab41-w1-int-3.fits']

In [62]:
three_color_params = [
    {
        'url': url,
        'Title': "3 color image"
    } for url in rgb_urls]

fc.show_fits_3color(three_color_params=three_color_params,
                    plot_id='3color-img')

{'success': True}

In [63]:
# Because we added more images, align and lock them by WCS again
fc.align_images(lock_match=True)

{'success': True}

## Show any data file
To explore any data file you have at hand without delving into image or table specific options, you can use the `show_data` method of the FireflyClient object.

### Preview metadata
It's often useful to preview the metadata of a data file you have no information about. `preview_metadata` parameter allows you to do that:

In [ ]:
some_fpath = '/Users/jsinghal/dev/cm/__test_data/MF.20210502.18830.fits'

In [ ]:
fc.show_data(some_fpath, preview_metadata=True)

{'success': True}

This allows you to select which extensions/HDUs to load from a multi-extension FITS file. Same can be done for a VOTable with multiple tables.

### Directly view data

In [ ]:
spectrum_file_url = 'http://irsa.ipac.caltech.edu/api/spectrumdm/convert/euclid/q1/SIR/102160339/EUC_SIR_W-COMBSPEC_102160339_2024-11-05T16:26:34.614296Z.fits?dataset_id=euclid_combspec&hdu=1644'

In [ ]:
fc.show_data(spectrum_file_url, preview_metadata=False)

{'success': True}

Note that show_data is a recent addition to firefly_client and is still evolving. It doesn't allow as much customization as image/table specific methods yet, but it is a quick way to get a look at your data.